<a href="https://colab.research.google.com/github/juanjo1228/Proyecto-IA/blob/main/04_preprocesado_y_modelo_XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import zipfile
import pandas as pd
import io
from google.colab import files
import os

print("Sube tu archivo ZIP...")
uploaded = files.upload()

# Obtener el nombre del ZIP
zip_filename = list(uploaded.keys())[0]
print("📁 Archivo ZIP cargado:", zip_filename)

# Guardarlo temporalmente
with open(zip_filename, "wb") as f:
    f.write(uploaded[zip_filename])

# Crear carpeta donde extraer
extract_folder = "train_zip_content"
os.makedirs(extract_folder, exist_ok=True)

print("🔓 Descomprimiendo ZIP...")
with zipfile.ZipFile(zip_filename, "r") as z:
    z.extractall(extract_folder)

print("✔ ZIP descomprimido.")

# Buscar archivos CSV dentro del ZIP
csv_files = [f for f in os.listdir(extract_folder) if f.endswith(".csv")]

if not csv_files:
    raise ValueError("❌ No se encontró ningún archivo CSV dentro del ZIP.")

print("📄 Archivos CSV encontrados:", csv_files)

# Cargar el primer CSV encontrado
csv_path = os.path.join(extract_folder, csv_files[0])
print("📥 Cargando:", csv_path)

# Intentar diversas codificaciones
encodings = ["utf-8", "latin-1", "ISO-8859-1", "cp1252"]
df = None

for enc in encodings:
    try:
        df = pd.read_csv(csv_path, encoding=enc)
        print(f"✔ CSV leído correctamente con codificación: {enc}")
        break
    except:
        print(f"❌ Falló con codificación: {enc}")

if df is None:
    raise ValueError("❌ No se pudo leer el CSV con ninguna codificación.")

print("\nColumnas del dataframe:")
print(df.columns.tolist())

df.head()



Sube tu archivo ZIP...


Saving train.zip to train (2).zip
📁 Archivo ZIP cargado: train (2).zip
🔓 Descomprimiendo ZIP...
✔ ZIP descomprimido.
📄 Archivos CSV encontrados: ['train.csv']
📥 Cargando: train_zip_content/train.csv
✔ CSV leído correctamente con codificación: utf-8

Columnas del dataframe:
['building_id', 'meter', 'timestamp', 'meter_reading']


,building_id,meter,timestamp,meter_reading
0,0,0,2016-01-01 00:00:00,0.0
1,1,0,2016-01-01 00:00:00,0.0
2,2,0,2016-01-01 00:00:00,0.0
3,3,0,2016-01-01 00:00:00,0.0
4,4,0,2016-01-01 00:00:00,0.0


In [8]:
# Eliminar duplicados
df = df.drop_duplicates()

# Detectar columnas numéricas y categóricas
num_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_cols = df.select_dtypes(include=['object']).columns.tolist()

print("Columnas numéricas:", num_cols)
print("Columnas categóricas:", cat_cols)

# Rellenar faltantes
df[num_cols] = df[num_cols].fillna(df[num_cols].mean())

for c in cat_cols:
    df[c] = df[c].fillna(df[c].mode()[0])

# Convertir categóricas a códigos (XGBoost friendly)
for c in cat_cols:
    df[c] = df[c].astype('category').cat.codes


Columnas numéricas: ['building_id', 'meter', 'meter_reading']
Columnas categóricas: ['timestamp']


In [9]:
TARGET = "meter_reading"


In [10]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=[TARGET])
y = df[TARGET]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

X_train.shape, X_test.shape


((16172880, 3), (4043220, 3))

In [11]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

model = XGBRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=8,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="reg:squarederror",
    tree_method="hist"
)

print("Entrenando XGBoost...")
model.fit(X_train, y_train)

# Predicción
y_pred = model.predict(X_test)

# Métricas
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) ** 0.5
r2 = r2_score(y_test, y_pred)

print("\n========== RESULTADOS XGBOOST ==========")
print("MAE:", mae)
print("RMSE:", rmse)
print("R²:", r2)


Entrenando XGBoost...

========== RESULTADOS XGBOOST ==========
MAE: 7277.704848632695
RMSE: 144539.32304198117
R²: 0.16522885156686162


In [12]:
model.save_model("xgb_model.json")
print("✔ Modelo guardado como xgb_model.json")
files.download("xgb_model.json")


✔ Modelo guardado como xgb_model.json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
model.save_model("xgb_model.model")
print("✔ Modelo guardado como xgb_model.model")
files.download("xgb_model.model")


✔ Modelo guardado como xgb_model.model


/usr/local/lib/python3.12/dist-packages/xgboost/sklearn.py:1115: UserWarning: [05:54:57] WARNING: /workspace/src/c_api/c_api.cc:1575: Saving model in the UBJSON format as default.  You can use a file extension: `json` or `ubj` to choose between formats.
  self.get_booster().save_model(fname)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>